In [110]:
# Import definition
from scipy.io import arff
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [111]:
# ------------------ PREPROCESSING AND DATA FILTERING -------------------- #

# Loading dataset into working desk
data = arff.loadarff('../data/breast.w.arff')
df = pd.DataFrame(data[0])

# Removes NaN values from dataset by deleting rows
df.dropna(axis=0, how="any", inplace=True)

# Gets X (data matrix) and y (target values column matrix)
X = df.drop("Class", axis=1).to_numpy()
y = df["Class"].to_numpy()

# Performs some preprocessing by turning labels into binaries (benign is 1)
# We are doing a "double conversion" to convert everything to Binary type
for count, value in enumerate(y):
    if value == b"benign":
        y[count] = "yes"
    else:
        y[count] = "no"
lb = LabelBinarizer()
y = lb.fit_transform(y)


In [112]:
# Holds predicted values for mlp without (1) and with early stopping (2)
predicted_1=[]
predicted_2=[]

# ------------------ QUESTION 2 ------------------------ #

# TODO: falar com o professor sobre o alpha (l2 regularization)

# Creates a 5 fold cross validator
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# For each train/test set, we use a MLP classifier without early stopping
for train_index, test_index in skf.split(X, y):

    # Uses indexes to fetch which values are going to be used to train and test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Scaling data (assuming standard scaling)
    #scaler = StandardScaler().fit(X_train)
    #X_train = scaler.transform(X_train)
    #X_test = scaler.transform(X_test)

    #MLP classifier without early stopping (L2)
    mlp = MLPClassifier(hidden_layer_sizes=(3, 2), early_stopping= False, alpha= 1, activation='relu', solver='sgd', learning_rate='constant', learning_rate_init=0.001, max_iter=2000, random_state=0)

    #Trains mlp classifier
    mlp.fit(X_train, y_train.ravel())

    #Predicts new y and saves new values
    y_predicted = mlp.predict(X_test)
    predicted_1.append(y_predicted)

# For each tindicator matrixrain/test set, we use a MLP classifier with early stopping
for train_index, test_index in skf.split(X, y):

    # Uses indexes to fetch which values are going to be used to train and test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Scaling data (assuming standard scaling)
    #scaler = StandardScaler().fit(X_train)
    #X_train = scaler.transform(X_train)
    #X_test = scaler.transform(X_test)

    #MLP classifier without early stopping (L2)
    mlp = MLPClassifier(hidden_layer_sizes=(3, 2), early_stopping=True, alpha= 1, activation='relu', solver='sgd', learning_rate='constant', learning_rate_init=0.001, max_iter=2000, random_state=0)

    #Trains mlp classifier
    mlp.fit(X_train, y_train.ravel())

    #Predicts new y and saves new values
    y_predicted = mlp.predict(X_test)
    predicted_2.append(y_predicted)


out1 = np.concatenate(predicted_1).ravel()
out2 = np.concatenate(predicted_2).ravel()

print('Confunsion Matrix without early stopping')
print(confusion_matrix(y.ravel(), out1))
tn, fp, fn, tp = confusion_matrix(y.ravel(), out1).ravel()
print('Accuracy:', (tn+tp)/(tn+tp+fn+fp))
print('Confunsion Matrix with early stopping')
print(confusion_matrix(y.ravel(), out2))
tn, fp, fn, tp = confusion_matrix(y.ravel(), out2).ravel()
print('Accuracy:', (tn+tp)/(tn+tp+fn+fp))

Confunsion Matrix without early stopping
[[ 79 160]
 [164 280]]
Accuracy: 0.5256222547584187
Confunsion Matrix with early stopping
[[  1 238]
 [  2 442]]
Accuracy: 0.6486090775988287
